In [1]:
# load data

In [2]:
import pandas as pd

raw_fundamental = pd.read_csv("./data/cfs_fundamental.csv", index_col=0)
raw_ohlcv = pd.read_csv("./data/ohlcvs.csv", index_col=0, low_memory=False)
raw_stocks = pd.read_csv("./data/stocks.csv", index_col=0)

In [3]:
from st_fa_tools.fa_tools.v2.preproc import OhlcvPreProc, StockPreProc, FundamentalPreProc
from st_fa_tools.fa_tools.v2.preproc import FaPreProc

fundamental_preproc = FundamentalPreProc(raw_fundamental)
preproc_fundamental = fundamental_preproc()

ohlcv_preproc = OhlcvPreProc(raw_ohlcv)
prices = ohlcv_preproc.get_ma_prices(window=32)

info_preproc = StockPreProc(raw_stocks)
shares = info_preproc.get_shares()

In [4]:
fa_preproc = FaPreProc(preproc_fundamental, prices, shares)
fundamental = fa_preproc()
fundamental

,date,stock_code,factor,amount,reprt_no
1,2020-11-25,950180,current_assets,2.762729e+10,2
2,2020-11-25,950180,fixed_assets,3.659419e+09,2
3,2020-11-25,950180,total_assets,3.128671e+10,2
4,2020-11-25,950180,current_liabilities,1.614036e+09,2
5,2020-11-25,950180,fixed_liabilities,4.289400e+08,2
...,...,...,...,...,...
11611,2024-02-28,207760,shares,8.033487e+07,0
11612,2024-02-28,011070,shares,2.366711e+07,0
11613,2024-02-29,004430,shares,2.400000e+07,0
11614,2024-02-29,018310,shares,1.470000e+07,0


In [5]:
from st_fa_tools.fa_tools.v2.proc import FaProc

fa_proc = FaProc(fundamental)

fa_0 = fa_proc("0")
fa_1 = fa_proc("1")
fa_2 = fa_proc("2")

In [6]:
# TPBR이 감소 한다는 것은 기대치가 낮아지고 있다는 것
tpbr_df = pd.concat(
    [
        fa_0["TPBR"].rename("recent_1"),
        fa_1["TPBR"].rename("recent_2"),
        fa_2["TPBR"].rename("recent_3"),
    ],
    axis=1,
)
_stock_codes_1 = tpbr_df[
    (tpbr_df["recent_1"] < tpbr_df["recent_2"]) & (tpbr_df["recent_2"] < tpbr_df["recent_3"])
].index

In [7]:
# 매년 당기순이익이 증가
net_profit_df = pd.concat(
    [
        fa_0["net_profit"].rename("recent_1"),
        fa_1["net_profit"].rename("recent_2"),
        fa_2["net_profit"].rename("recent_3"),
    ],
    axis=1,
)
_stock_codes_2 = net_profit_df[
    (net_profit_df["recent_3"] < net_profit_df["recent_2"])
    & (net_profit_df["recent_2"] < net_profit_df["recent_1"])
].index

In [8]:
# 유동 BPS가 price 이하
_stock_codes_3 = fa_0[fa_0["CBPS"] > fa_0["price"]].index

# 유동부채 비율 0.5 이하
_stock_codes_4 = fa_0[fa_0["C_debt_ratio"] < 0.5].index

In [9]:
fa_filtered_codes = (
    set(_stock_codes_1) & set(_stock_codes_2) & set(_stock_codes_3) & set(_stock_codes_4)
)
len(fa_filtered_codes)

17

In [10]:
from st_ta_tools.ta_tools.models.relative_strength import SimpleRelativeStrengthModel

In [11]:
results = dict()
RS_CFG = {"window": 30}
for stock_code in fa_filtered_codes:
    prices = raw_ohlcv[raw_ohlcv["stock_code"] == stock_code]["close"].rename("price")
    simple_relative_strength_model = SimpleRelativeStrengthModel(prices, RS_CFG)
    srs_indicator = simple_relative_strength_model.indicator()
    srs_signal = simple_relative_strength_model.signal(srs_indicator, continious=True, trend=False)
    result = {
        "recent_10": srs_signal["signal"].tail(10).mean().round(5),
        "recent_30": srs_signal["signal"].tail(30).mean().round(5),
        "recent_60": srs_signal["signal"].tail(60).mean().round(5),
    }
    results[stock_code] = result

In [12]:
techinical_df = pd.DataFrame(results).T
techinical_df

,recent_10,recent_30,recent_60
002810,-0.44572,-0.30278,-0.13453
006200,0.04107,-0.11522,-0.11930
094970,-0.23789,-0.27245,-0.13613
020000,0.00491,0.03293,-0.01431
111770,-0.18563,-0.06295,0.07845
078140,0.07941,0.08576,0.01402
029460,-0.18565,-0.25486,-0.26918
002920,-0.22367,0.03459,-0.01248
005680,-0.06665,-0.25078,-0.30311
008110,-0.01087,0.06695,0.22053


In [13]:
ta_filtered_codes = techinical_df[
    (techinical_df["recent_10"] > 0)
    | (techinical_df["recent_30"] > 0)
    | (techinical_df["recent_60"] > 0)
].index
len(ta_filtered_codes)

10

In [14]:
from st_pa_tools.pa_tools.generators.highlow_based_generator import HighLowBasedPositionGenerator
from st_pa_tools.pa_tools.models.position_rank_index import PositionRankIndexModel

hlb_pg = HighLowBasedPositionGenerator()
pri_model = PositionRankIndexModel()

In [15]:
from tqdm import tqdm

In [16]:
time_size = 90
position_size = 300
results = dict()
for stock_code in tqdm(ta_filtered_codes):
    single_ohlcv = raw_ohlcv[raw_ohlcv["stock_code"] == stock_code]
    single_ohlcv.columns = [col.lower() for col in single_ohlcv.columns]
    #
    highs = single_ohlcv["high"]
    lows = single_ohlcv["low"]
    volumes = single_ohlcv["volume"]
    #
    position = hlb_pg.get_time_dependent_volume_position(
        highs, lows, volumes, time_size, position_size
    )
    price = single_ohlcv["close"].iloc[-1]
    pri = pri_model.calc(price, position)
    results[stock_code] = pri

100%|██████████| 10/10 [00:00<00:00, 11.67it/s]


In [17]:
position_df = pd.DataFrame([results], index=["pri"]).T
position_df

,pri
006200,0.91
020000,0.81
111770,0.92
078140,0.95
002920,0.63
008110,0.77
088790,0.01
046310,0.93
000850,0.31
010060,0.90


In [18]:
pa_filtered_stocks = position_df[(0.6 < position_df["pri"]) & (position_df["pri"] < 0.8)].index

In [19]:
corps = pd.read_csv("./data/corps.csv", index_col=0)
corps["stock_code"] = corps["stock_code"].apply(lambda x: str(x).zfill(6))

stocks = pd.read_csv("./data/stocks.csv", index_col=0)
stocks["stock_code"] = stocks["stock_code"].apply(lambda x: str(x).zfill(6))

In [20]:
info = pd.merge(left=corps, right=stocks.drop(columns=["stock_nm"]), on=["stock_code"])

In [21]:
final_df = info[info["stock_code"].isin(pa_filtered_stocks)].sort_values(
    "market_cap", ascending=False
)
final_df

,stock_code,stock_nm,sector,product,market,shares,market_cap
295,002920,유성기업,자동차 신품 부품 제조업,"철도차량부품(피스턴링,실린더라이너,발브가이드,타펫트),자동차부품,주강,주물 제조,판매",KOSPI,25947500,75118012500
241,008110,대동전자,통신 및 방송 장비 제조업,"통신음향,전자기계기구용 플라스틱제품 제조,도매",KOSPI,10490447,70810517250


In [22]:
fa_0[fa_0.index.isin(pa_filtered_stocks)].loc[:, ["CBPS", "TBPS", "price"]].astype(int)

factor,CBPS,TBPS,price
stock_code,,,
002920,5210,12611,2875
008110,9276,21447,8559


In [23]:
buying_stocks = list(set(final_df[final_df["market"] == "KOSPI"]["stock_code"]))

In [24]:
def get_latest_price(ohlcvs,stock_code):
    latest_price = ohlcvs[ohlcvs['stock_code'] == stock_code]['close'].iloc[-1]
    return latest_price

In [25]:
buying_stock_price = [
    (stock_code, get_latest_price(ohlcvs=raw_ohlcv, stock_code=stock_code) * 1.05)
    for stock_code in buying_stocks
]
buying_stock_price

[('008110', 7161.0), ('002920', 3055.5)]

In [26]:
import pickle

with open("./data/buying_stock_price.pkl", "wb") as f:
    pickle.dump(buying_stock_price, f)